In [1]:
# Copyright 2021 NVIDIA Corporation. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# ================================

<img src="http://developer.download.nvidia.com/compute/machine-learning/frameworks/nvidia_logo.png" style="width: 90px; float: right;">

## Two-Stage Recommender Systems

In large scale recommender systems pipelines, the size of the item catalog (number of unique items) might be in the order of millions. At such scale, a typical setup is having two-stage pipeline, where a faster candidate retrieval model quickly extracts thousands of relevant items and a then a more powerful ranking model (i.e. with more features and more powerful architecture) ranks the top-k items that are going to be displayed to the user. For ML-based candidate retrieval model, as it needs to quickly score millions of items for a given user, a popular choices are models that can produce recommendation scores by just computing the dot product the user embeddings and item embeddings. Popular choices of such models are **Matrix Factorization**, which learns low-rank user and item embeddings, and the **Two-Tower architecture**, which is a neural network with two MLP towers where both user and item features are fed to generate user and item embeddings in the output.


### Dataset

In this notebook, we are building a Two-Tower model for Item Retrieval task using synthetic datasets that are mimicking the real [Ali-CCP: Alibaba Click and Conversion Prediction](https://tianchi.aliyun.com/dataset/dataDetail?dataId=408#1) dataset.
### Learning objectives
- Preparing the data with NVTabular
- Training and evaluating Two-Tower model with Merlin Models
- Exporting the model for deployment

### Importing Libraries

In [2]:
import os
os.environ["TF_GPU_ALLOCATOR"]="cuda_malloc_async"
import cudf

import nvtabular as nvt
from nvtabular.ops import *
from merlin.models.utils.example_utils import workflow_fit_transform

from merlin.schema.tags import Tags

import merlin.models.tf as mm
from merlin.io.dataset import Dataset

import tensorflow as tf

2022-04-05 16:17:19.854912: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-04-05 16:17:20.952592: I tensorflow/core/common_runtime/gpu/gpu_process_state.cc:214] Using CUDA malloc Async allocator for GPU: 0
2022-04-05 16:17:20.952717: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 16254 MB memory:  -> device: 0, name: Quadro GV100, pci bus id: 0000:15:00.0, compute capability: 7.0


In [3]:
# disable INFO and DEBUG logging everywhere
import logging
logging.disable(logging.WARNING)

### Feature Engineering with NVTabular

Let's generate synthetic train and validation dataset objects.

In [4]:
from merlin.datasets.synthetic import generate_data

DATA_FOLDER = os.environ.get("DATA_FOLDER", "/workspace/data/")
train, valid = generate_data("aliccp-raw", 1000000, set_sizes=(0.7, 0.3))

In [5]:
# define output path for the processed parquet files
output_path = os.path.join(DATA_FOLDER, "processed")

We keep only positive interactions where clicks==1 in the dataset with `Filter()` op.

In [6]:
user_id = ["user_id"] >> Categorify() >> TagAsUserID()
item_id = ["item_id"] >> Categorify() >> TagAsItemID()

item_features = ["item_category", "item_shop", "item_brand"] >> Categorify() >> TagAsItemFeatures()

user_features = ['user_shops', 'user_profile', 'user_group', 
       'user_gender', 'user_age', 'user_consumption_2', 'user_is_occupied',
       'user_geography', 'user_intentions', 'user_brands', 'user_categories'] \
        >> Categorify() >> TagAsUserFeatures()

inputs = user_id + item_id + item_features + user_features + ['click'] 

outputs = inputs >> Filter(f=lambda df: df["click"] == 1)

With `transform_aliccp` function, we can execute fit() and transform() on the raw dataset applying the operators defined in the NVTabular workflow pipeline above. The processed parquet files are saved to output_path.

In [7]:
from merlin.datasets.ecommerce import transform_aliccp
transform_aliccp((train, valid), output_path, nvt_workflow=outputs)

/usr/local/lib/python3.8/dist-packages/cudf/core/dataframe.py:1253: UserWarning: The deep parameter is ignored and is only included for pandas compatibility.
  warnings.warn(


## Building a Two-Tower Model with Merlin Models

We will use Two-Tower Model for item retrieval task. Real-world large scale recommender systems have hundreds of millions of items (products) and users. Thus, these systems often composed of two stages: candidate generation (retrieval) and ranking (scoring the retrieved items). At candidate generation step, a subset of relevant items from large item corpus is retrieved. You can read more about two stage Recommender Systems here. In this example, we're going to focus on the retrieval stage.

A Two-Tower Model consists of item (candidate) and user (query) encoder towers. With two towers, the model can learn representations (embeddings) for queries and candidates separately. 

<img src="./images/TwoTower.png"  width="30%">

Image Adapted from: [Off-policy Learning in Two-stage Recommender Systems](https://dl.acm.org/doi/abs/10.1145/3366423.3380130)

We use the `schema` object to define our model.

In [8]:
output_path

'/workspace/data/processed'

In [9]:
train = Dataset(os.path.join(output_path, 'train', '*.parquet'))
valid = Dataset(os.path.join(output_path, 'valid', '*.parquet'))

schema = train.schema

In [10]:
schema = schema.select_by_tag([Tags.ITEM_ID, Tags.USER_ID, Tags.ITEM, Tags.USER])

We can print out the feature column names.

In [11]:
schema.column_names

['user_id',
 'item_id',
 'item_category',
 'item_shop',
 'item_brand',
 'user_shops',
 'user_profile',
 'user_group',
 'user_gender',
 'user_age',
 'user_consumption_2',
 'user_is_occupied',
 'user_geography',
 'user_intentions',
 'user_brands',
 'user_categories']

We expect the label names to be empty.

In [12]:
label_names = schema.select_by_tag(Tags.TARGET).column_names
label_names

[]

### Negative sampling
Many datasets for recommender systems contain implicit feedback with logs of user interactions like clicks, add-to-cart, purchases, music listening events, rather than explicit ratings that reflects user preferences over items. To be able to learn from implicit feedback, we use the general (and naive) assumption that the interacted items are more relevant for the user than the non-interacted ones.
In Merlin Models we provide some scalable negative sampling algorithms for the Item Retrieval Task. In particular, we use in this example the in-batch sampling algorithm which uses the items interacted by other users as negatives within the same mini-batch.

### Building the Model

Now, let's build our Two-Tower model. In a nutshell, we aggregate all user features to feed in user tower and feed the item features to the item tower. Then we compute the positive score by multiplying the user embedding with the item embedding and sample negative items (read more about negative sampling [here](https://openreview.net/pdf?id=824xC-SgWgU) and [here](https://medium.com/mlearning-ai/overview-negative-sampling-on-recommendation-systems-230a051c6cd7)), whose item embeddings are also multiplied by the user embedding. Then we apply the loss function on top of the positive and negative scores.

In [13]:
model = mm.TwoTowerModel(
    schema,
    query_tower=mm.MLPBlock([128, 64], no_activation_last_layer=True),        
    loss="categorical_crossentropy",  
    samplers=[mm.InBatchSampler()],
    embedding_options = mm.EmbeddingOptions(infer_embedding_sizes=True),
    metrics=[mm.RecallAt(10), mm.NDCGAt(10)]
)

Let's explain the parameters in the TwoTowerModel():
- no_activation_last_layer: when set True, no activation is used for top hidden layer. Learn more [here](https://storage.googleapis.com/pub-tools-public-publication-data/pdf/b9f4e78a8830fe5afcf2f0452862fb3c0d6584ea.pdf).
- infer_embedding_sizes: when set True, automatically defines the embedding dimension from the feature cardinality in the schema

**Metrics:**

The following information retrieval metrics are used to compute the Top-10 accuracy of recommendation lists containing all items:

- **Normalized Discounted Cumulative Gain (NDCG@10)**: NDCG accounts for rank of the relevant item in the recommendation list and is a more fine-grained metric than HR, which only verifies whether the relevant item is among the top-k items.

- **Recall@10**: Also known as HitRate@n when there is only one relevant item in the recommendation list. Recall just verifies whether the relevant item is among the top-n items.

We need to initialize the dataloaders.

In [14]:
model.set_retrieval_candidates_for_evaluation(train)
model.compile(optimizer='adam', run_eagerly=False)
model.fit(train, validation_data=valid, batch_size=4096, epochs=3)

2022-04-05 16:17:25.278668: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


Epoch 1/3
85/86 [============================>.] - ETA: 0s - recall_at_10: 0.0177 - ndcg_10: 0.0088 - loss: 8.3001 - regularization_loss: 0.0000e+00 - total_loss: 8.3001

2022-04-05 16:17:42.029046: W tensorflow/core/grappler/optimizers/loop_optimizer.cc:907] Skipping loop optimization for Merge node with control input: cond/then/_0/cond/cond/branch_executed/_185


86/86 [==============================] - 17s 110ms/step - recall_at_10: 0.0177 - ndcg_10: 0.0089 - loss: 8.2735 - regularization_loss: 0.0000e+00 - total_loss: 8.2735 - val_recall_at_10: 0.0252 - val_ndcg_10: 0.0142 - val_loss: 7.7711 - val_regularization_loss: 0.0000e+00 - val_total_loss: 7.7711
Epoch 2/3
86/86 [==============================] - 7s 85ms/step - recall_at_10: 0.0263 - ndcg_10: 0.0192 - loss: 8.2724 - regularization_loss: 0.0000e+00 - total_loss: 8.2724 - val_recall_at_10: 0.0245 - val_ndcg_10: 0.0202 - val_loss: 7.7719 - val_regularization_loss: 0.0000e+00 - val_total_loss: 7.7719
Epoch 3/3
86/86 [==============================] - 7s 81ms/step - recall_at_10: 0.0226 - ndcg_10: 0.0188 - loss: 8.2719 - regularization_loss: 0.0000e+00 - total_loss: 8.2719 - val_recall_at_10: 0.0203 - val_ndcg_10: 0.0164 - val_loss: 7.7722 - val_regularization_loss: 0.0000e+00 - val_total_loss: 7.7722


## Exporting Retrieval Models

So far we have trained and evaluated our Retrieval model. Now, the next step is to deploy our model and generate top-K recommendations given a user (query). We can efficiently serve our model by indexing the trained item embeddings into an **Approximate Nearest Neighbors (ANN)** engine. Basically, for a given user query vector, that is generated passing the user features into user tower of retrieval model, we do an ANN search query to find the ids of nearby item vectors, and at serve time, we score user embeddings over all indexed top-K item embeddings within the ANN engine.

In doing so, we need to export
 
- user (query) tower
- item and user features
- item embeddings

#### Save User (query) tower

We are able to save the user tower model as a TF model to disk. The user tower model is needed to generate a user embedding vector when a user feature vector <i>x</i> is fed into that model.

In [15]:
query_tower = model.retrieval_block.query_block()
query_tower.save('query_tower')

#### Extract and save User features

With `unique_rows_by_features` utility function we can easily extract both unique user and item features tables as cuDF dataframes. Note that for user features table, we use `USER` and `USER_ID` tags.

In [16]:
from merlin.models.utils.dataset import unique_rows_by_features
user_features = unique_rows_by_features(train, Tags.USER, Tags.USER_ID).compute().reset_index(drop=True)

In [17]:
user_features.head()

user_id  user_shops  user_profile  user_group  user_gender  user_age  \
0        1           1             1           1            1         1   
1        2           2             1           1            1         1   
2        3           3             1           1            1         1   
3        4           4             1           1            1         1   
4        5           5             1           1            1         1   

   user_consumption_2  user_is_occupied  user_geography  user_intentions  \
0                   1                 1               1                1   
1                   1                 1               1                2   
2                   1                 1               1                3   
3                   1                 1               1                4   
4                   1                 1               1                5   

   user_brands  user_categories  
0            1                1  
1            2                2  
2            3                3  
3            4                4  
4            5                5

In [18]:
user_features.shape

(646, 12)

In [19]:
# save to disk
user_features.to_parquet('user_features.parquet')

#### Extract and save Item features

In [20]:
item_features = unique_rows_by_features(train, Tags.ITEM, Tags.ITEM_ID).compute().reset_index(drop=True)

In [21]:
item_features.head()

item_id  item_category  item_shop  item_brand
0        1              1          1           1
1        2              2          2           2
2        3              3          3           3
3        4              4          4           4
4        5              5          5           5

In [22]:
# save to disk
item_features.to_parquet('item_features.parquet')

#### Extract and save Item embeddings

In [23]:
item_embs = model.item_embeddings(Dataset(item_features, schema=schema), batch_size=1024)
item_embs_df = item_embs.compute(scheduler="synchronous")

In [24]:
item_embs_df

item_id  item_category  item_shop  item_brand         0         1  \
0          1              1          1           1  0.119024  0.023623   
1          2              2          2           2  0.006361  0.003338   
2          3              3          3           3  0.074537  0.001323   
3          4              4          4           4  0.153636 -0.080778   
4          5              5          5           5 -0.011768  0.018682   
..       ...            ...        ...         ...       ...       ...   
630      743            743        743         743  0.064798 -0.065997   
631      749            749        749         749 -0.008370  0.082715   
632      752            752        752         752 -0.109725  0.286749   
633      753            753        753         753 -0.048022  0.033861   
634      761            761        761         761  0.061938  0.029517   

            2         3         4         5  ...        54        55  \
0   -0.039086 -0.000039  0.070910 -0.005304  ...  0.085932 -0.102355   
1    0.124567 -0.069380  0.122115 -0.201470  ...  0.026831 -0.132266   
2    0.090352 -0.175259  0.179352 -0.021959  ... -0.026903 -0.045079   
3   -0.123449 -0.089321  0.139870  0.015942  ... -0.076668 -0.113005   
4   -0.025512 -0.085963 -0.134499 -0.186269  ...  0.061904 -0.109536   
..        ...       ...       ...       ...  ...       ...       ...   
630 -0.031717 -0.067496 -0.004143 -0.014273  ...  0.060530 -0.230824   
631  0.117570  0.019689  0.087900 -0.079214  ...  0.023555 -0.114855   
632 -0.070979 -0.030827 -0.008561 -0.039951  ... -0.021042 -0.058173   
633 -0.245225 -0.045610  0.033465 -0.101318  ... -0.165817  0.000217   
634 -0.072761 -0.101128  0.080592 -0.028033  ... -0.049143 -0.149057   

           56        57        58        59        60        61        62  \
0   -0.058122 -0.088331  0.059050 -0.025986 -0.023549  0.159494  0.311440   
1    0.129241 -0.088970 -0.108118  0.021614 -0.081004  0.124680  0.345408   
2    0.189090  0.133597 -0.008829 -0.157224  0.050934  0.161178  0.242915   
3    0.069237  0.238331  0.085635 -0.083087  0.018781  0.139706  0.167688   
4    0.095276  0.032400 -0.116568  0.061201  0.125158 -0.042238  0.208102   
..        ...       ...       ...       ...       ...       ...       ...   
630  0.015961  0.040368 -0.074561 -0.002969 -0.199503  0.121633  0.207420   
631 -0.063390  0.111346 -0.263790  0.061215  0.143212  0.041604  0.319031   
632 -0.141470 -0.028167 -0.035519 -0.103719  0.089092  0.027004  0.279829   
633  0.043540 -0.102218 -0.152996 -0.059092  0.099988 -0.010920  0.136724   
634  0.153742 -0.037752 -0.049190 -0.039576  0.111076  0.125575  0.472521   

           63  
0   -0.172660  
1   -0.209228  
2   -0.130376  
3   -0.141418  
4   -0.167115  
..        ...  
630 -0.342876  
631 -0.119142  
632 -0.315001  
633  0.016220  
634 -0.132032  

[635 rows x 68 columns]

In [25]:
# select only embedding columns
item_embeddings = item_embs_df.iloc[:, 4:]

In [26]:
item_embeddings.head()

0         1         2         3         4         5         6  \
0  0.119024  0.023623 -0.039086 -0.000039  0.070910 -0.005304 -0.046084   
1  0.006361  0.003338  0.124567 -0.069380  0.122115 -0.201470 -0.095038   
2  0.074537  0.001323  0.090352 -0.175259  0.179352 -0.021959 -0.197854   
3  0.153636 -0.080778 -0.123449 -0.089321  0.139870  0.015942 -0.062343   
4 -0.011768  0.018682 -0.025512 -0.085963 -0.134499 -0.186269 -0.103873   

          7         8         9  ...        54        55        56        57  \
0 -0.071140  0.053499  0.149303  ...  0.085932 -0.102355 -0.058122 -0.088331   
1  0.078551  0.133017 -0.013526  ...  0.026831 -0.132266  0.129241 -0.088970   
2  0.132899  0.135168  0.017511  ... -0.026903 -0.045079  0.189090  0.133597   
3 -0.102844  0.021416  0.109875  ... -0.076668 -0.113005  0.069237  0.238331   
4  0.053713  0.100448  0.005378  ...  0.061904 -0.109536  0.095276  0.032400   

         58        59        60        61        62        63  
0  0.059050 -0.025986 -0.023549  0.159494  0.311440 -0.172660  
1 -0.108118  0.021614 -0.081004  0.124680  0.345408 -0.209228  
2 -0.008829 -0.157224  0.050934  0.161178  0.242915 -0.130376  
3  0.085635 -0.083087  0.018781  0.139706  0.167688 -0.141418  
4 -0.116568  0.061201  0.125158 -0.042238  0.208102 -0.167115  

[5 rows x 64 columns]

In [27]:
# save to disk
item_embeddings.to_parquet('item_embeddings.parquet')

That's it. You have learned how to train and evaluate your Two-Tower retrieval model, and then how to export the required components to be able to deploy this model to generate recommendations. In order to learn more on serving a model to [Triton Inference Server](https://github.com/triton-inference-server/server), please explore the examples in the [Merlin](https://github.com/NVIDIA-Merlin/Merlin) and [Merlin Systems](https://github.com/NVIDIA-Merlin/systems) repos.